In [1]:
from playwright.sync_api import sync_playwright
import json
import datetime
from bs4 import BeautifulSoup
import re



def extract_text(page, selector):
    elements = page.query_selector_all(selector)
    print(page.content())
    return "\n".join([element.inner_text() for element in elements if element.inner_text().strip()])

def run(playwright,topic='Technology'):
    browser = playwright.chromium.launch(headless=False,executable_path='C:\Program Files\Google\Chrome\Application\chrome.exe')
    context = browser.new_context()
    
    # 打开 BBC 新闻主页
    page = context.new_page()
    context.set_extra_http_headers({
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.190 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'Accept-Language': 'en-US,en;q=0.9',
        'Accept-Encoding': 'gzip, deflate, br',
        # 可以根据需要添加其他头部信息
    })
    page.goto("https://finance.yahoo.com/")

    # 获取所有新闻链接
    input()
    # news_links = page.query_selector_all("a[slot='full-post-link']")
    # urls = [link.get_attribute("href") for link in news_links if link.get_attribute("href")]
    # print('urls:',urls)

    html_source = page.content()
    question_count_soup = BeautifulSoup(html_source, 'html.parser')
    # all_question_htmls = question_count_soup.find_all('a', {'class': 'q-box qu-display--block qu-cursor--pointer qu-hover--textDecoration--underline Link___StyledBox-t2xg9c-0 dxHfBI'}, href=True)import re
    all_question_htmls = question_count_soup.find_all('a', class_=re.compile(r'q-box'), href=True)
    all_question_htmls=[x for x in all_question_htmls if 'answer_timestamp' in x['class']]
    # all_question_htmls[4]['class']
    # ['q-box', 'Link___StyledBox-t2xg9c-0', 'dFkjrQ', 'answer_timestamp', 'qu-cursor--pointer', 'qu-hover--textDecora...-underline']
    question_count = len(all_question_htmls)
    if question_count is None or question_count == 0:
        print('topic does not have questions...')
        # continue
        return None
    soup = BeautifulSoup(html_source, 'html.parser')

    all_htmls = soup.find_all('a', {'class': 'q-box qu-display--block qu-cursor--pointer qu-hover--textDecoration--underline Link___StyledBox-t2xg9c-0 dxHfBI'}, href=True)
    question_count_after_scroll = len(all_htmls)
    print(f'number of questions for this topic : {question_count_after_scroll}')
    url_bag=[]
    # add questions to a set for uniqueness
    for html in all_htmls:
        url_bag.add(html['href'])

    sleep_time = (round(random.uniform(15, 32), 1))
    # page.wait_for_timeout(sleep_time * 1000)

    # 遍历链接并提取信息
    for url in url_bag[20:26]:#
        # endswith a number:
        # if not 'news/articles/' in url:#TODO add more content type #url[-1].isdigit():
        #     print("not articles skip",url)
        #     continue
        full_url=f"https://www.reddit.com{url}"
        print("url:",full_url)
        page.goto(full_url)
        text_blocks=None

        # 提取页面标题和文本
        # title = page.query_selector("h1")  # 假设标题总是在 h1 标签中
        # content = page.query_selector("article")  # 假设主要内容在 article 标签中
        try:
            text_blocks = extract_text(page, "div[slot='text-body'] div div p")#, ul[class=]
            print("text_blocks:",text_blocks)
            title = extract_text(page, "h1[slot='title']")
            print("title:",title)
            text_blocks=title+'\n'+text_blocks

            # element=page.locator("section[data-component='text-block']")
            # text_blocks = element.inner_text()
            error=False


        except Exception as e:  
            print(e)
            print("can not find text_blocks")
            error=True


        entry = {"date": datetime.datetime.now().strftime("%Y-%m-%d-%H-%M"), "error": error, "url": full_url,'text_blocks':text_blocks}
        with open("reddit_data.jsonl", "a", encoding="utf-8") as file:
            # for entry in data:
            json.dump(entry, file, ensure_ascii=False)
            file.write("\n")

        print("\n----------\n")

    # 关闭浏览器
    context.close()
    browser.close()

with sync_playwright() as playwright:
    run(playwright)


Error: It looks like you are using Playwright Sync API inside the asyncio loop.
Please use the Async API instead.

In [1]:
# from playwright.sync_api import Playwright, sync_playwright, expect


# def run(playwright: Playwright) -> None:
#     browser = playwright.chromium.launch(headless=False)
#     context = browser.new_context()
#     page = context.new_page()
#     page.goto("https://www.bbc.com/news")
#     page.get_by_role("link", name="A volcano spews lava and").click()

#     # ---------------------
#     context.close()
#     browser.close()


# with sync_playwright() as playwright:
#     run(playwright)


from playwright.sync_api import sync_playwright
import json
import datetime
def extract_text(page, selector):
    elements = page.query_selector_all(selector)
    print(page.content())
    return "\n".join([element.inner_text() for element in elements if element.inner_text().strip()])

def run(playwright):
    browser = playwright.chromium.launch(headless=False)
    context = browser.new_context()
    
    # 打开 BBC 新闻主页
    page = context.new_page()
    page.goto("https://www.bbc.com/news")

    # 获取所有新闻链接
    news_links = page.query_selector_all("a[data-testid='internal-link']")
    urls = [link.get_attribute("href") for link in news_links if link.get_attribute("href")]
    print('urls:',urls)

    # 遍历链接并提取信息
    for url in urls:#[20:26]
        # endswith a number:
        if not url[-1].isdigit():
            print("not endswith a number skip",url)
            continue
        full_url=f"https://www.bbc.com{url}"
        print("url:",full_url)
        page.goto(full_url)
        text_blocks=None

        # 提取页面标题和文本
        # title = page.query_selector("h1")  # 假设标题总是在 h1 标签中
        # content = page.query_selector("article")  # 假设主要内容在 article 标签中
        try:
            text_blocks = extract_text(page, "section[data-component='text-block']")#, ul[class=]
            print("text_blocks:",text_blocks)

            # element=page.locator("section[data-component='text-block']")
            # text_blocks = element.inner_text()
            error=False


        except Exception as e:  
            print(e)
            print("can not find text_blocks")
            error=True


        entry = {"date": datetime.datetime.now().strftime("%Y-%m-%d-%H-%M"), "error": error, "url": full_url,'text_blocks':text_blocks}
        with open("BBCdata.jsonl", "a", encoding="utf-8") as file:
            # for entry in data:
            json.dump(entry, file, ensure_ascii=False)
            file.write("\n")

        print("\n----------\n")

    # 关闭浏览器
    context.close()
    
    browser.close()


with sync_playwright() as playwright:
    run(playwright)


Error: It looks like you are using Playwright Sync API inside the asyncio loop.
Please use the Async API instead.

In [4]:
# ls=['/news/wealthy-net-worth-considered-poor-190014440.html', 'https://a.beap.gemini.yahoo.com/mbclk?bv=3.0.0&es=AQAAAF9i7xYgBFBYABOnN5DKyk6KygeeBcCPfFwnrOtC8FQyinKlgmm1hYgCidfglhU9KOlMR2xZkdrA927CNFeMc_KMuVUKdvv8fYAmLbfpZG0x3kjkx4CRlbniYbJ1pLsUoSsIkMQPTo7TCdqDuAkzQz5Rtyp9sPTdJaGtXc1AJLBMEODf3uq7SyZ-S_K6WwY0G-sWObOF2S3TgjKZqdH9vWvGY5a4-PAdNQHxsMDiRfOZL4L0ImH-xI71FG0sj5geTrokOhI7PaDSaxau5kuyhDSWhp14Ojp8bxhdp5UGVm_5hDXyJZ9atlzaEzyXBuluKbCzaGM8O3Fd-7dEvijNyWXTwps5l8BriQEQR_eeejZDVtHRCd89MGg_MRuJnbIa-xTA45ecrRX0q9c4pf89O_P9yA-q6Qe8WA3nz4BJkBtTF33OG4jy6naw268gdVQoh9-kfquqgldsESsfM-KfRmDimYP1vTd7QCzwX-_fbYdEI7UTEHFF1TpN-CKaRL8kDkyZJ3TWFe8gDTNqxnAHf7bnbgOl1_fxub7G3_yEXaK_4FWYtPkjVXprMuoxHDTZHCVdsLdVhyYPHrDr4k7LfcPQckBMbzQiISACRK33Z3r9fj7etXEuYuXGdn8W-54SPVcW5_vWeAL_xgiY3G3Fwf9jfSwywFEWZETK0b5gCGXLRLYH6MQieiZ1WsEiQkzwIKiJhq5zUxUKA7ITFCaIBzC0ke1WX9LZP-0iXu9st9NpXwlDyO-rMxhZShegQud5-qGt0MZRo2zDUS19nrT4fTTNfOxXtNxZAKIrdVqTIFnMuN2GEe9m1vuITCFj4H9h0sLWC_nzJag3z7LJDpKAmg0FY8QsuR71_wvSzlt4r5cIOwDpWC8UQKApKE8sKrkmOSJf0OxICV5be8RUnWG-hAQQnrPCcrb6qvrIL4A14k637w756JBU0eNSmXVQm1jd1KGCEJXffHgtCGMCbOUHxgkjzOMiBMHX_hGwLY_zNuEj7jPAB9fbaOBMSeixN2QZoULSxYHyI3Yns5JzcW7lS6KJZcGikd4EOhudNhgWbCAMOUjC8icAR4UjLIpbcP4GArLbzWylKdQTm74qLIKB-obCvi46Du57pY_KsAUbfX5tlxS95Kj26OzW6n_W4bbzwlRpXPhp4volIf72ZyN79QzgypPw7IZqMgEi77ga8NoLXpIYKqlkTRZDsSBBxdE8miThlZ9S51NQ2bC3xG-4BPYkiJt6RSX-w2oBuXQXO6RYiapO1yIF1Y59oxxmwQ5B89f_OzrU7-8gOFi8mv6NA8sK_yQ_QZj64ZS4cFmY0Et7ccmA6ve6Uch4ZpLYfilZRPJ54QS289tzpPu2IMNzrbQgrdSwR5u3lH7mv12_0oD8gJxXEH0bSSy4QgqRUpSKeV-jq-cOBYNDTr_DwehOxP7UeBHKqMKNq1TzEx8m_XW4LKD9Oia7WExB0DaWDG2ZQzMsAjymov8pglGM2gGW3zhtYqzc_Ykcb92pRXylYYZzz6e3RIKMMyTTZ_AZu7GKDcnMXBboQxRUjCfhg7fky0sJ024kfURPZuKImouT4TXbR0KeB65wF0gecw9cywVS5C9JqKwc-Qg76QBwy4yu5ljKoXKFSRaE96ADNdHTcQp1nfKmUmAu5iMmmsTCaIRUzJQEns7fbcBC9fEVm1_-BioazAdxSfD1tJgHabuHlGpnmrsbUcQM7P-aFbIAl_YDMNNGJqNOJZYfLbc_z-Vf79Zfp_28ZHvDCMlyk0Gf6rMa9uPoelwBBDNnigZ9wZLPvVaglKE5sGWOwBjhZYnHYbiDMCon4hHZLF7SPt1AVMeHsIfwnDb1s3hhxnCQ-9gQjLp0tS15TMhzRk0c2C-b1Q5O490HO4ENUrjJrr9PVFkR4KWMkbOVqDA2UNTVbJKLQnBmbef4ZdfLzG4whOe_HZuPdegmx0bbAazMnWS78HNzHsIRoLe0VbpXYQmBv6ZE622Qllhh1PejYCwJ6DibenYzTHht20wE12SHZ-ARguQcXMNytJfoLJzCaozONF1hoCNdb_m7gbVj3qrFUTPZBmjfyQgBRkkBX7ohXHYYCOFXZCjjqPHpxRTVsQcnaH33vjhNLP41hNmh9yeUWaIxwZOUyqVzo_rUzOg-lUHioc--wRD7ZO2yTgVoQ4EiQBuuAjjigeq98l_9dGOitlTqRkCpIadML3zmq8LvN14s5M8asb_-cbLb--cZjQmbaMTBAPPItZHjB2YJOm16pmKigIb5wJHukTBFCXC2IQsjJSoycyNJGISE4X9KO1A5B-eqmgFz5L8obxb0s_QOwB8WP3xCCpIzROTupmHushIrKpettqxLV2hiMoPgEc1KZd8t_ODJ3ZoO1fuU25ocILihSOvBHg3jci_kWew3_hbYLQRixQtm3ZJ8DbX8utKSXM1SmXZfIMJdmO0Wv9wPM2RRxRRYJYjqnj1ydPRcgXXWjAkJ14ltqeJTa0tXCf_ta1SU93-Jv2IBGUUmivJMM8cECrq3VqeXTzSOebATkcN2-fiJnjcJuMO2qGFp2kBCKsBeyKuSmZHAhGxg_xmQgCLqPb_k8vra2Ge60DRWzJdXYqvvPcU4N5MDMmKahp6b0FqB9xWVDE-hkoX9JB30UpG4q0K4-O3DyZv9SzwLRjLIIXT4U8sOuMkLjg5Qrt5dvLzYAkOIud4GJDGl8cCG23iAgL697i4Ru9wRuo9MUtcOWyQboDFOYnx0MUMMDYnoBMzZ_gxkY7OfRTVYY5mHhG9MMnR2XVmYHz4oMc7eISkYRk2l89K-4J6ehlIlnY3uxbTF0EZTQAOANPxDsqNY70QTqLHPTkmBxS3uVXBQoXQlU2yEPKmPPz-uB2d-t3924WKFzN6i1xeL5mO0RXu4qOdTVZ4TpI5t-e0aLLYhy9Q2482T3FpzhKXfc6uaIyPmpmFzxg574Utlw3oG1g899kaLMlA1wchpVWY', '/news/much-cash-1-million-annuity-200010067.html', '/news/bill-gates-34-billion-invested-120000483.html', '/news/hyperloop-one-shut-down-failing-191602007.html', '/news/warren-buffett-really-likes-1-110700418.html', 'https://a.beap.gemini.yahoo.com/mbclk?bv=3.0.0&es=AQAAAPeav3XpT8xC4rVaPhbN129B-BZsLjNqY653j73C2Wv2vj3AI48rbqgc4U_fIyhC3ASyuWEfWIx7F7-hevIcy_Mqi32s7dRcvbBC1KIF57ba1kqWJSVSkba6rfahcNVtpqOKbvhFd7ISxGSdZGTTMSyLwGIxHXQFyLujE6dsFWQgnEWufm74D8gjek1TrCUREWbSpvm9qC7hJIsHeKkTqc4U39niuAndKbbVConMq1ulFvY0brr_Mq9YLQQ5EFDokbOtU6YAkFkL9hvJouOkLMVFW8txk9tOcgl-RuieURL-MKZOl6-OD91WIDwxLBARK_42L84dxl3Xil613ZlpN9NnpsFGQD0lcHrDHEJ-sLeSt9HObM4Pk1KuWQpH-ak4sEex66VF4xIaJROY8vivevk2VcGc-yuRItzn-NBE0bcp_qTty71MNhlQ86bHrmFOV0lRrngUhMRvDy6MKwkKWyVjsHPMd9xyHO5mSJK1PAB6i_EqBoQ-Wv86-8LVgWy3bFDyOFxQgIu8HDU9rubkcuaBkCzfmVp2sJK8c2iQatfJvtrstv6Hj0Op7x9uWMM3oZv2MxZouXi_t56h1nGPd-XEaMAxrQU44nMNjenD1_kgZI0tBwHB9xCA3Q0Op_ZKJI0osNPHR-pYPcMNOKzweWRX8z2qFDxvubrPNnN8fO4jPING52_oLLwsTD3PX6gM-pwp_SsH6UkKWYZgkzYIBoA5uC6zNHUEa2LvYTSn0LXrBVqXC2PYyL4zq_PItHy5QmMnnsEO-ImVEmidM9Mp55hJHeDoZFR-y0PtW1oQowLJhYO_INA92dw12QtJpKXONzGqaLA4UYaN4sDHz_7rNaE30v10jIpo50Vjwz9WUrNhoFPsb04ZyLsJVe11I2zCDq1Jn3ijbJCwHksj_zfK', 'https://finance.yahoo.com/research/reports/MS_1200638_StockPicks_1703288709000?yptr=yahoo&ncid=yahooproperties_plusresear_nm5q6ze1cei', '/news/no-one-wants-used-evs-050011036.html', '/news/german-property-prices-plummet-housing-194050106.html', '/news/forget-nvidia-buy-magnificent-artificial-115200637.html', 'https://a.beap.gemini.yahoo.com/mbclk?bv=3.0.0&es=AQAAAI_iPoFRlr63wGwmpNAbt3IwsVunbwVTSdcnimV_IjwZrbPrGNJa6pjEMdij9i9ozL24scPu4hu1t20CYUUXp_rYvo1GyFMaBfyvDjx_PPKAMHFfR1x9FlOnVyDDRVPCVrNWI5H59vJmF--n41ALyGAvv7XAsXs1V5s7LI1D011ygP_FiNBadeqaWkDF_cQQITfpphc-9uZU7DBHTU3YVuKrBbnODWFQnQllopW71Zlw7PNq7WMfs0Moz7PBscnCR_fH6DRJF5rbjVk2rVAM6dxuLoPLbNzeExmzvj-65HvFk4sUtU6UBNrPhqwbrRSddi68NNzyOADaVNhXlSu-9NqvcQytq264gRZ5kcIAGCAuzy4Tg95C4s3K4w7e81eeGqmX1JE3mXZV4pPR4W-IEuyoE-MqrNGVAcs_oh2JlOwRxHVbgM6YNs1O5MUQOZsZ3ByxPcLkQjO5BTbHHsYL3hKDmPbB6UXUl_grfcQjuAf9I4RsfrvvTjBS-W6_PZ6pdRbSDF9Mh2VlacJGnhMFfAYHEuyHG_-KEg8E7K6v4d9eYX6aZOl5DRIHXow3A5dB4zqbHIC0e_CtUdcHAI5C5p4QGahmT5g4iV3kt17AFT7AZg1Jw5-S6s_-8ZhnkurMjsagm4LdjQZwpw9MbxEUIE_YcSaMQVwz_SRqGV6BbMUksPhkufF65i9rfUBRVtAcXrBtqWP6JFcqQzGTF5xyjv5WjvIr5raIAetpAYZGIMfCG3GtIKDT4zybHYZN-L7p_j2OfdHwbIbpzn2u3haFsKBbTAombaiRvXpEG_yE6L0yooFG6q7mAF6JShUghrSLf57mqCYpknEZ00CApPO6wjZW5mwauQ', '/news/trump-now-complaining-soaring-stocks-214539045.html']

# for i in ls:
#     print('https://finance.yahoo.com/'+i)

import re
def clean_special_char(text = "your text with special characters like \u200b\u200c\n\t."):
# 
    # Regular expression to match unwanted characters
    pattern = r'[\u200b\u200c\n\t]+'
    cleaned_text = re.sub(pattern, '', text)
    return cleaned_text

clean_special_char()

'your text with special characters like .'